In [1]:
from datasets import load_dataset, load_metric
from transformers import pipeline

raw_dataset = load_dataset('cnn_dailymail', '3.0.0')

classifier = pipeline("summarization")

/opt/homebrew/Caskroom/miniconda/base/envs/NLP-summarization/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Reusing dataset cnn_dailymail (/Users/jeroen/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234)
100%|████████████████████████████████████████████| 3/3 [00:00<00:00, 161.67it/s]
No model was supplied, defaulted to t5-small (https://huggingface.co/t5-small)
2022-04-01 16:46:45.371865: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at t5-small.
If your tas

In [2]:
def show_examples(dataset, num_samples=3, seed=42):
    samples = dataset["train"].shuffle(seed=seed).select(range(num_samples))
        
    for idx, sample in enumerate(samples):
        display(f'sample {idx}: {sample["article"]} \n')
        display(f'highlight {idx}: {sample["highlights"]} \n')
        display(f'id: {sample["id"]}')
        display('-------')
        
def get_samples(dataset, num_samples=10):
    return dataset["train"].shuffle(seed=1).select(range(num_samples))

def get_random_sample(dataset):
    sample = dataset["train"].shuffle(seed=1).select(range(1)) 
    return [sample["article"][0], sample["highlights"][0]]

In [3]:
show_examples(raw_dataset, 1, 23)
# get_random_sample(raw_dataset)

Loading cached shuffled indices for dataset at /Users/jeroen/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234/cache-17d3e0cea13717ef.arrow


'sample 0: WASHINGTON (CNN) -- The National Transportation Safety Board began four days of hearings Tuesday on how to stem the "drastic increase" in medical helicopter accidents. Smoke rises from Spectrum Health Butterworth Hospital in Grand Rapids, Michigan, after a helicopter crash in May. Over a recent 12-month period, the board probed nine fatal medical helicopter accidents that killed 35 people, a development that one board member called "alarming." Medical helicopters "provide an important service to the public" in swiftly transporting ill and injured patients and donor organs, the board said on its Web site. Chopper pilots must operate "safely and quickly" in bad weather, at night or on "unfamiliar landing sites," the board added. "This hearing will be extremely important because it can provide an opportunity to learn more about the industry so that possibly we can make further recommendations that can prevent these accidents and save lives," said Robert Sumwalt, chairman of the

'highlight 0: Transportation safety board beginning four days of hearings .\nBoard examines reported "drastic increase" in accidents and deaths .\nNine air ambulance crashes killed 35 people during one-year period .\nBoard\'s 2006 safety recommendations not fully implemented, it says . \n'

'id: 4929e54ae3f6711b4bd8da27a46d0f8a90c3b3bf'

'-------'

### Model description
BART is a transformer encoder-encoder (seq2seq) model with a bidirectional (BERT-like) encoder and an autoregressive (GPT-like) decoder. BART is pre-trained by (1) corrupting text with an arbitrary noising function, and (2) learning a model to reconstruct the original text.

BART is particularly effective when fine-tuned for text generation (e.g. summarization, translation) but also works well for comprehension tasks (e.g. text classification, question answering).

In [4]:
from transformers import AutoTokenizer

# A text-to-text transformer from Google
# https://ai.googleblog.com/2020/02/exploring-transfer-learning-with-t5.html
pretrained_model_name = "t5-small"

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)

In [5]:
article, highlight = get_random_sample(raw_dataset)
print(tokenizer.tokenize(article))

Loading cached shuffled indices for dataset at /Users/jeroen/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234/cache-d65aa25a11484207.arrow
Token indices sequence length is longer than the specified maximum sequence length for this model (798 > 512). Running this sequence through the model will result in indexing errors


['▁The', '▁daughter', '▁of', '▁football', '▁legend', '▁Colin', '▁He', 'n', 'dry', '▁has', '▁spoken', '▁of', '▁her', '▁disgust', '▁at', '▁the', '▁Government', '’', 's', '▁failure', '▁to', '▁regulate', '▁cosmetic', '▁surgeon', 's', '▁', '–', '▁more', '▁than', '▁', 'a', '▁decade', '▁after', '▁her', '▁mother', '▁suffered', '▁bot', 'ched', '▁lip', 'o', 's', 'u', 'ction', '▁that', '▁led', '▁to', '▁her', '▁death', '.', '▁Den', 'is', 'e', '▁He', 'n', 'dry', '▁died', '▁in', '▁2009', '▁aged', '▁42', '▁from', '▁an', '▁infection', '▁she', '▁caught', '▁during', '▁surgery', '▁to', '▁rebuild', '▁her', '▁stomach', '.', '▁She', '▁had', '▁been', '▁in', '▁and', '▁out', '▁of', '▁hospital', '▁for', '▁seven', '▁years', '▁after', '▁', 'a', '▁plastic', '▁surgeon', '▁punct', 'ure', 'd', '▁her', '▁', 'bowel', '▁nine', '▁times', '▁in', '▁', 'a', '▁routine', '▁lip', 'o', 's', 'u', 'ction', '▁operation', '▁in', '▁2002', '.', '▁An', 'gu', 'ish', ':', '▁Colin', '▁and', '▁Den', 'is', 'e', '▁He', 'n', 'dry', '▁with', 

In [6]:
metric = load_metric("rouge")
metric

Metric(name: "rouge", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each predictions
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLSum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/datasets/issues/617
    use_stemmer: Bool indicating whether Porter stemmer should be used to strip word suffixes.
    use_agregator: Return aggregates if this is set to True
Retu

In [7]:
max_input_length = 1024
max_target_length = 128

prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["article"]]
    
    model_inputs = tokenizer(examples["article"], max_length=max_input_length, truncation=True)
    
    # Set up the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlights"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

Use 🤗 Datasets map function to apply the preprocessing function over the entire dataset. You can speed up the map function by setting batched=True to process multiple elements of the dataset at once:

In [9]:
tokenized_cnn_dailymail = raw_dataset.map(preprocess_function, batched=True)

100%|███████████████████████████████████████████| 12/12 [00:06<00:00,  1.80ba/s]


In [ ]:
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

model = TFAutoModelForSeq2SeqLM.from_pretrained(pretrained_model_name)
model

In [ ]:
batch_size = 8
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 1

model_name = model_checkpoint.split("/")[-1]
push_to_hub_model_id = f"{model_name}-finetuned-xsum"